In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt


import tensorflow as tf
import keras as kr

import datetime
import matplotlib.dates as mdates

import seaborn as sns
sns.set(palette="Set2")

minute_stock_data_OCT2010 = np.genfromtxt('mquote201010.csv', delimiter=',',names=True,dtype=None)


Using Theano backend.


## Return to Index 
- [Index](http://141.142.208.248:8000/notebooks/BW-Project/Index%20Page.ipynb)

## Data Sorting/Retrieving Functions

Dates should be entered in in the following format: yearmonthday. Example Oct 1 2010 --> 20101001. Stock symbols should be entered in as strings corresponding to the list from the variables section. 

pull_stock_from_symbol(stock_data,symbol)
    
    returns index of where the first occurence of symbol is located in the orginal data array
    
pull_stock_from_index(stock_data,index)

    returns all the data from the orginal data array. This is not filtered for just the values.
    
pull_stock_from_date_and_symbol(stock_data,symbol,date)

    returns market values for a specific date (given that it is not a weekend) for a specific stock
    
pull_all_data_in_month(stock_data,symbol)

    return market values for full month given specfi symbol

In [2]:
def pull_stock_from_symbol(stock_data,symbol):
    #definition to pull specific index of stock data given symbol
    for i in range(minute_stock_data_OCT2010_length):
        if(minute_stock_data_OCT2010[i][0].decode("utf-8")==symbol):
            return i
        else:
            i+=stock_steps
    return False

In [3]:
def pull_stock_from_index(stock_data,index):
    #definition to pull specific data from index
    if(index%21==0 and index<minute_stock_data_OCT2010_length and index >=0):
        return minute_stock_data_OCT2010[index:index+21]
    else:
        index = index-(index%21)
        return minute_stock_data_OCT2010[index:index+21]

In [4]:
def pull_stock_from_date_and_symbol(stock_data,symbol,date):
    index = pull_stock_from_symbol(stock_data,symbol)
    single_day_data = np.zeros(shape=(391,),dtype=float)
    for i in range(index+21):
        if(minute_stock_data_OCT2010[i][1]==date):
            for x in range(391):
                single_day_data[x]=minute_stock_data_OCT2010[i][3+x]
        
    return single_day_data


In [5]:
def pull_all_data_in_month(stock_data,symbol):
    start_index = pull_stock_from_symbol(stock_data,symbol)
        #Get the correct start index for stock
    symbol_month_data = np.zeros(shape=(8211,),dtype=np.float)
        #Create an empty array of correct size
        #Fill it in with a double array
    index = 0 
    for x in range (start_index,start_index+21):
        for y in range(3,394):
            symbol_month_data[index]=minute_stock_data_OCT2010[x][y]
            #print(symbol_month_data[index])
            index+=1

    #print(index)  
    return symbol_month_data

In [6]:
def time_array_per_day(date):
    date_format = str(date)
    date_format = date_format[0:4]+'-'+date_format[4:6]+'-'+date_format[6:8]
    date_start = date_format+'T09:30'
    date_end = date_format+'T16:01'
    return np.arange(date_start, date_end, dtype='datetime64[m]')
    

## Graphing Functions

graph_single_day_stock_data(symbol,date)

    Symbol inputted as str and date entered in in the formal yearmonthday ex(20101001)
    Prints out opening market day price and closing market day price and highest market value
    
graph_full_month_data(symbol)

    Symbol inputted as str. Prints the highest market and lowest market value for the month graphs the values
    of the stock over the course of the month
  
graph_find_correlation(symbol1,symbol2)

    Symbol1 and symbol2 inputted as str. Graphs a scatter plot along with a line of best fit between two stocks.
    Prints out correlation coefficient R in addition to the slope of the best fit line.
    
graph_all_days_in_month(symbol)

    Symbol inputted as str, graphs subplots of each individual day in the month for the corresponding symbol.
    
    
    

In [7]:
def graph_single_day_stock_data(symbol,date):
    stock_to_graph = pull_stock_from_date_and_symbol(minute_stock_data_OCT2010,symbol,date)
    single_date_data = time_array_per_day(date)
    fig_stock,ax_stock=plt.subplots(figsize=(15,10))
    ax_stock.plot(single_date_data,stock_to_graph,color='indigo')
    ax_stock.set_xlabel('Hours of Current Day')
    ax_stock.set_ylabel('Stock Values')
    ax_stock.set_title('Stock Data for Single Day')
    print("Opening Day Value:",stock_to_graph[0],"Closing Day Value:",stock_to_graph[390])
    print("Highest Market Value For Day:",np.amax(stock_to_graph))
    print("Lowest Market Value For Day:",np.amin(stock_to_graph))

In [8]:
def graph_full_month_data(symbol):
    stock_to_graph =pull_all_data_in_month(minute_stock_data_OCT2010,symbol)
    #daily = mdates.DayLocator()
    #hours = mdates.HourLocator()
    #dfmt = mdates.DateFormatter('%b %d')
    fig_stock,ax_stock=plt.subplots(figsize=(15,10))
    plt.tight_layout()
    ax_stock.plot(stock_to_graph,color='indigo')
    #plt.gcf().autofmt_xdate()
    #ax_stock.set_xlim(734046.395833,734074.666667)
    #ax_stock.xaxis.set_minor_locator(daily)
    ax_stock.set_xticklabels([time for time in time_stamps[::len(time_stamps)//10]]);
    ax_stock.set_xlabel('Month-Day-Year of Oct.')
    ax_stock.set_ylabel('Stock Values')
    ax_stock.set_title('Full Month Stock Data')
    print("Highest Market Value For Month:",np.amax(stock_to_graph))
    print("Lowest Market Value For Month:",np.amin(stock_to_graph))

In [9]:
def graph_find_correlation(symbol1,symbol2):
    stock1 = pull_all_data_in_month(minute_stock_data_OCT2010,symbol1)
    stock2 = pull_all_data_in_month(minute_stock_data_OCT2010,symbol2)
    fig_stock,ax_stock = plt.subplots(figsize=(15,10))
    ax_stock.scatter(stock1,stock2,color='indigo')
    ax_stock.plot(np.unique(stock1), np.poly1d(np.polyfit(stock1, stock2, 1))(np.unique(stock1)),color='orchid')
    slope_bestfit,intercept=np.polyfit(stock1,stock2,1)
    ax_stock.set_xlabel('Stock 1 Values')
    ax_stock.set_ylabel('Stock 2 Values')
    ax_stock.set_title('Looking for Correlations')
    A = np.vstack([stock1,np.ones(len(stock1))]).T
    m,c = np.linalg.lstsq(A,stock2)[0]
    #plt.plot(stock1,stock1*m+c,color='blue')
    correlation_r = np.corrcoef(stock2,stock1)[1,0]
    print("Slope of Best fit is equal to:",slope_bestfit)
    print("Numpy Correlation Coefficient is equal to:",correlation_r)
    print("The slope  of best fit and the numpy correlation coefficient are for full month of data.")


In [10]:
def graph_all_days_in_month(symbol):
    index = pull_stock_from_symbol(minute_stock_data_OCT2010,symbol)
    
    fig,ax = plt.subplots(3,7,figsize=(35,17))
    for x in range(3):
        for y in range(7):
            date = minute_stock_data_OCT2010[index][1]
            ax[x][y].plot(time_array_per_day(date),pull_stock_from_date_and_symbol(minute_stock_data_OCT2010,symbol,date),color='indigo')
            xfmt = mdates.DateFormatter('%H:%M')
            ax[x][y].xaxis.set_major_formatter(xfmt)
            ax[x][y].set_title('Stock for Data:' + str(minute_stock_data_OCT2010[index][1]))
            index+=1
            #xloc = plt.MaxNLocator(3)
            #ax[x][y].xaxis.set_major_locator(xloc)
            
    plt.suptitle("Full Month Plotted as Individual Days",fontsize=20)
    fig.text(0.5, 0.07, 'Opening (9:30) to Closing (4:00)', ha='center')
    fig.text(0.1, 0.5, 'Stock Values', va='center', rotation='vertical')

In [11]:
def bayesian_correlation(symbol1,symbol2):
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, xlabel=symbol1, ylabel=symbol2)
    colors = np.linspace(0.1, 1, len(stock_A_data))
    mymap = plt.get_cmap("winter")
    sc = ax.scatter(pull_all_data_in_month(minute_stock_data_OCT2010,symbol1), pull_all_data_in_month(minute_stock_data_OCT2010,symbol2), c=colors, cmap=mymap, lw=0)
    cb = plt.colorbar(sc)
    cb.ax.set_yticklabels([time for time in time_stamps[::len(time_stamps)//10]]);
    return sc

In [12]:
graph_full_month_data('A')


NameError: name 'minute_stock_data_OCT2010_length' is not defined

In [ ]:
graph_all_days_in_month('A')

In [ ]:
graph_single_day_stock_data('A',20101029)

In [ ]:
graph_find_correlation('A','AA')

In [1]:
minute_stock_data_OCT2010_length = len(minute_stock_data_OCT2010) 
    #number of elements within the array
stock_steps = 21 
    #number of indices between separate stocks
start_day_index = 3 
    #where first minute starts
end_day_index = 393 
    #where last minute ends

time_stamps_temp = np.arange('2010-10-01T09:30', '2010-10-29T16:01', dtype='datetime64[m]')
count = 0
indexes = np.zeros(shape=(11520,),dtype=np.int)
for i in range(len(time_stamps_temp)):
    current_date = (str(time_stamps_temp[i]))[0:10]
    if(np.is_busday(current_date)==False):
         indexes[count]=int(i)
         count+=1
    else:
        pass
time_stamps = np.delete(time_stamps_temp,indexes)


num_over = 0
indexes_num2 = np.zeros(shape=(20980,),dtype=int)
for x in range(len(time_stamps)):        
    if('16:01' in str(time_stamps[x])):
        for y in range(1049):
            indexes_num2[num_over]=(x+y)
            num_over+=1

time_stamps = np.delete(time_stamps,indexes_num2)

time_stamps_real = np.zeros(shape=(8211,),dtype='datetime64[m]')
#Method instead just append np arrange on to the end of the thingies.

num_min_in_buisnness_month = len(time_stamps)
    #Number of minute level data points per the october month.

NameError: name 'minute_stock_data_OCT2010' is not defined